## Minimal transversals

Let $\mathcal{S}$ be a family of sets. A set $T$ is a *transversal* (or *hitting set*) of $\mathcal{S}$ if $T$ intersects every $S\in\mathcal{S}$. The family of minimal transversals of $\mathcal{S}$ is denoted by  $\text{Tr}(\mathcal{S})$.

The minimality of $T$ can be characterized by the following condition:
If $x\in T$, then $S\cap T= \{x\}$ for some $S\in\mathcal{S}$. 

In [ ]:
"""
Author: Nandor Sieben (https://github.com/nandorsieben)
"""

import cpmpy as cp
from typing import List

In [7]:
def Tr(sets:List):
    """
    Given a family of sets, returns all minimal transversals.
    """
    X = {s for S in sets for s in S} # all elements across all sets, deduplicated
    
    # Decision Variables
    t = {x:cp.boolvar() for x in X} # whether to inlude an element into the transversal

    # Model
    model = cp.Model()

    # Constraints
    # 1) transversal
    for S in sets:
        model += cp.any([t[s] for s in S]) # must hit each set by intersecting with at least one element
    # 2) minimal transversal
    #       for each element x, if it is included in our transversal T, 
    #       there must exist a set S which intersects with T in only that single element x
    for x in X: 
        model += t[x].implies(
            cp.any(
              [cp.all([~t[s] for s in S if s!=x]) 
                for S in sets if x in S]
              )
            )
    
    # Solve and collect results
    result = []
    model.solveAll(display=lambda: result.append({a for a in t if t[a].value()}))
    return result 

In [8]:
Tr([{1,2},{2,3}])

[{2}, {1, 3}]

In [9]:
Tr([{2},{1,3}])

[{2, 3}, {1, 2}]

In [10]:
Tr([{1,2},{2,4,5},{2,3,5},{4,7,8}])

[{1, 4, 5}, {1, 3, 4}, {1, 5, 8}, {1, 5, 7}, {2, 7}, {2, 8}, {2, 4}]